In [1]:
import random
import pandas as pd
from codecs import open
import json

In [2]:
def load_set(file_name):
    to_out = []
    with open(file_name, 'r', encoding="utf-8") as file:
        for line in file:
            to_out.append(line)
    return to_out

def load_titles(file_name):
    to_out = []
    with open(file_name, 'r', encoding="utf-8") as file:
        for line in file:
            content = json.loads(line)
            title = content['wiki_title'] + '\n'
            to_out.append(title)
    return to_out

        
def save_set(file_name, str_list):
    with open(file_name, 'wb') as file:
        for line in str_list:
            file.write('{}'.format(line).encode('utf-8'))

#index_input = load_set('../../data/extractive_stage/input_with_punc.csv2000.index')
#tfidf_input = load_set('../../data/extractive_stage/input_with_punc.csv2000.tfidf')
#cheating_input = load_set('../../data/extractive_stage/input_with_punc.csv2000.cheating')
#inputs = load_set('../../data/wikisum_ptbr/input_with_punc.csv')
#titles = load_titles('../../data/wikisum_ptbr/docids_with_punc.json')
#target = load_set('../../data/wikisum_ptbr/output_with_punc.csv')


In [3]:
print(len(index_input), len(tfidf_input), len(cheating_input), len(target))
df = pd.DataFrame({'title' : titles, 'index':index_input, 'tfidf':tfidf_input, 'cheating':cheating_input, 'target':target})

114652 114652 114652 114652


In [8]:
train_valid=df.sample(frac=0.9,random_state=200) #random state is a seed value
test=df.drop(train_valid.index)

In [9]:
print(len(train_valid))
print(len(test))

103187
11465


In [10]:
save_set('../../data/extractive_stage/train_with_punc_title', train_valid['title'])
save_set('../../data/extractive_stage/train_with_punc_index', train_valid['index'])
save_set('../../data/extractive_stage/train_with_punc_tfidf', train_valid['tfidf'])
save_set('../../data/extractive_stage/train_with_punc_cheating', train_valid['cheating'])
save_set('../../data/wikisum_ptbr/train_with_punc_target', train_valid['target'])
save_set('../../data/extractive_stage/test_with_punc_title', test['title'])
save_set('../../data/extractive_stage/test_with_punc_index', test['index'])
save_set('../../data/extractive_stage/test_with_punc_tfidf', test['tfidf'])
save_set('../../data/extractive_stage/test_with_punc_cheating', test['cheating'])
save_set('../../data/wikisum_ptbr/test_with_punc_target', test['target'])


In [13]:
# save testset inputs with max 500 words 
def limit_input(input_list, max_words=500, sep='</s>'):
    to_out = []
    for line in input_list:
        limited = ''
        sentences = line.split(sep)
        n = 0
        for sent in sentences:
            n = n + len(sent.split(' '))
            if n > max_words:
                break
            limited = limited + sent + sep
        limited = limited + '\n'
        to_out.append(limited)
    return to_out

#save_set('../../data/extractive_stage/500words_test_with_punc_index', limit_input(test['index']))
#save_set('../../data/extractive_stage/500words_test_with_punc_tfidf', limit_input(test['tfidf']))
#save_set('../../data/extractive_stage/500words_test_with_punc_cheating', limit_input(test['cheating']))


In [16]:
from datasets import load_metric

test_target = load_set('../../data/wikisum_ptbr/test_with_punc_target')

# load rouge
rouge = load_metric("rouge")

# calculate rouge1 recall for different tfidf sizes
N = [100*i for i in range(20)]
print('N, rouge1, rouge2, rougeL')
for n in N:
    limited_tfidf = limit_input(test['tfidf'], max_words=n)
    test_df = pd.DataFrame({'document' : limited_tfidf, 'summary' : test_target})
    results = rouge.compute(predictions=test_df["document"], references=test_df["summary"], rouge_types=["rouge1", "rouge2", "rougeL"])
    rouge1R = results["rouge1"].mid.recall
    rouge2R = results["rouge2"].mid.recall
    rougeLR = results["rougeL"].mid.recall
    # print rouge results
    print(n, rouge1R, rouge2R, rougeLR)

N, rouge1, rouge2, rougeL
0 0.0 0.0 0.0
100 0.18758376133914922 0.031462773768792834 0.12252644225453496
200 0.3190846821400905 0.05402911828563653 0.1964732414654826
300 0.3889992055850368 0.06992636932096458 0.23948925410026986
400 0.4332710837692165 0.0824890643426209 0.2694956541586987
500 0.46508473227742186 0.0933206278262835 0.29309370471197527
600 0.488806336771594 0.10248409227939825 0.31188606039569655
700 0.5080385689753523 0.11078497074642885 0.3277153922388222
800 0.5234194245756425 0.11789888892247812 0.3410507275947761
900 0.5365856134074828 0.12448861985050867 0.35308959254194383
1000 0.5478384572978274 0.1304767654442074 0.36365767757307366
1100 0.5551131772814504 0.13450104454827844 0.3703694577225141
1200 0.555246331881869 0.1345683678243751 0.3704637899319079
1300 0.5552791039594387 0.13457494747210375 0.37048980719186897
1400 0.5552822190301508 0.13458075167888023 0.3705038894628797
1500 0.5552865620180203 0.13458075167888023 0.370508984330826
1600 0.55529232931746

In [3]:
import random

random.seed(200)

# different way: first divide indexes and then divide files
indexes = [i for i in range(114652)]
random.shuffle(indexes)
n_train = int(len(indexes)*0.9)
train_indexes = indexes[0:n_train]
test_indexes = indexes[n_train:]
print(len(indexes), len(train_indexes) + len(test_indexes))


114652 114652


In [5]:
#inputs = load_set('../../data/wikisum_ptbr/input_with_punc.csv')
#titles = load_titles('../../data/wikisum_ptbr/docids_with_punc.json')
#target = load_set('../../data/wikisum_ptbr/output_with_punc.csv')

# input files
with open('../../data/wikisum_ptbr/input_with_punc.csv','r') as input_f:
    with open('../../data/wikisum_ptbr/docids_with_punc.json','r') as titles_f:
        with open('../../data/wikisum_ptbr/output_with_punc.csv','r') as output_f:
            #output files
            with open('../../data/wikisum_ptbr/train_test_split/input_train.csv','w') as train_input_f:
                with open('../../data/wikisum_ptbr/train_test_split/docids_train.json','w') as train_titles_f:
                    with open('../../data/wikisum_ptbr/train_test_split/output_train.csv','w') as train_output_f:
                        with open('../../data/wikisum_ptbr/train_test_split/input_test.csv','w') as test_input_f:
                            with open('../../data/wikisum_ptbr/train_test_split/docids_test.json','w') as test_titles_f:
                                with open('../../data/wikisum_ptbr/train_test_split/output_test.csv','w') as test_output_f:
                                    for i in range(len(indexes)):
                                        if(i%10000==0):
                                            print(i)
                                        input_line = input_f.readline()
                                        docids_line = titles_f.readline()
                                        output_line = output_f.readline()
                                        if(i in train_indexes):
                                            train_input_f.write(input_line)
                                            train_titles_f.write(docids_line)
                                            train_output_f.write(output_line)
                                        else:
                                            test_input_f.write(input_line)
                                            test_titles_f.write(docids_line)
                                            test_output_f.write(output_line)                                            


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
